Load jupyter sql extension


In [1]:
#Load Jupter SQL extension
%reload_ext sql
%config SqlMagic.autocommit = False

In [47]:
import pandas as pd

In [3]:
#Load database
%sql sqlite:///../data/raw/frantamarit.db

'Connected: @../data/raw/frantamarit.db'

In [4]:
%%sql tables <<

SELECT 
    name
FROM 
    sqlite_master 
WHERE 
    type ='table' AND 
    name NOT LIKE 'sqlite_%';

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable tables


In [5]:
%%sql master_join <<

SELECT
    rank_info.id,
    rank_info.name as fullName,
    rank_info.position,
    personal_info.lastName,
    personal_info.age,
    personal_info.gender,
    personal_info.country,
    personal_info.image,
    business_info.Source as source,
    business_info.worth,
    business_info.worthChange,
    business_info.realTimePosition
FROM 
    rank_info
LEFT JOIN 
    personal_info
ON
    rank_info.id = personal_info.id
LEFT JOIN
    business_info
ON 
    rank_info.id = business_info.id
ORDER BY
    rank_info.position DESC;

 * sqlite:///../data/raw/frantamarit.db
Done.
Returning data to local variable master_join


In [16]:
#Showing tables
df_master = master_join.DataFrame()

In [92]:
df_master

,id,fullName,position,lastName,age,gender,country,image,source,worth,worthChange,realTimePosition,sourceDetails
0,1690,olgun zorlu,2208.0,zorlu,54.0,2,turkey,https://specials-images.forbesimg.com/imageser...,diversified,1.0,NaN,1978.0,diversified
1,2599,zhuo jun,2207.0,zhuo,52.0,1,hong kong,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,1978.0,printed circuit boards
2,8536,zhu xingming,2206.0,zhu,51.0,0,china,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,0.0,1856.0,electrical equipment
3,5887,zhou liangzhang,2205.0,zhou,55.0,0,none,https://specials-images.forbesimg.com/imageser...,manufacturing,1.0,NaN,2134.0,electrical equipment
4,9216,zhao xiaoqiang,2204.0,zhao,51.0,2,china,https://specials-images.forbesimg.com/imageser...,fashion & retail,1.0,0.0,2134.0,"fashion, entertainment"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,8911,supaluck umpujh & family,NaN,None,NaN,0,None,None,NaN,NaN,NaN,NaN,NaN
2256,4098,thomas duff,NaN,None,NaN,0,None,None,NaN,NaN,NaN,NaN,NaN
2257,6885,zakhar smushkin,NaN,None,NaN,0,None,None,NaN,NaN,NaN,NaN,NaN
2258,5401,zhang zhongneng & family,NaN,None,NaN,0,None,None,NaN,NaN,NaN,NaN,NaN


In [87]:
#To use afterwards
def lowercase_feature_1(df,col):
    return df[col].str.lower()

In [91]:
to_lower_cols = ['lastName', 'fullName', 'country', 'source']

for i in to_lower_cols:
    df_master[i] = lowercase_feature_1(df_master, i)

In [90]:
df_master['fullName']

0                    olgun zorlu
1                       zhuo jun
2                   zhu xingming
3                zhou liangzhang
4                 zhao xiaoqiang
                  ...           
2255    supaluck umpujh & family
2256                 thomas duff
2257             zakhar smushkin
2258    zhang zhongneng & family
2259                   zhu yicai
Name: fullName, Length: 2260, dtype: object

In [26]:
def extract_str(df, col, regex):
    return df[col].str.extract(regex)

def extract_float(df, col, regex):
    return df[col].str.extract(regex).astype('float64')

def replace_str(df, col, x, y, rw):
    return df[col].replace(x, y, inplace=rw)

In [21]:
df_master['sourceDetails'] = extract_str(df_master,'source',r"> (.*)")

In [22]:
df_master['source'] = extract_str(df_master,'source',r"(.*)  =")

In [23]:
#In USD Billion
df_master['worth'] = extract_float(df_master,'worth',r"(\d+.?\d)")

In [24]:
#In USD Billion
df_master['worthChange'] = extract_float(df_master,'worthChange',r"(-?\d+.?\d*) ")

In [29]:
df_master['country'].replace("People's Republic of China", 'China',inplace=True)

In [41]:
#Using Float but better to be treated as integer later on
df_master['age'] = extract_float(df_master,'age',r'(\d+)')

In [44]:
df_master['age'] = [2017 - i if i > 1000 else i for i in df_master['age']]

In [46]:
df_master['age'].sort_values().unique()

array([19., 21., 23., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34.,
       35., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49.,
       50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62.,
       63., 64., 65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75.,
       76., 77., 78., 79., 80., 81., 82., 83., 84., 85., 86., 87., 88.,
       89., 90., 91., 92., 93., 94., 95., 96., 97., 99., nan])

In [20]:
#We don't want to get rid of the NaN yet, so we'll fill them with 0s.
def fill_0(x):
    return x.fillna(0, inplace=True)

fill_0(df_rank_info)
df_rank_info

,id,name,position
0,5542,anant asavabhokhin & family,0.0
1,4183,arash ferdowsi,0.0
2,4761,banthoon lamsam & family,0.0
3,1449,bill adderley & family,0.0
4,1624,boris zingarevich,0.0
...,...,...,...
2255,9216,zhao xiaoqiang,2204.0
2256,5887,zhou liangzhang,2205.0
2257,8536,zhu xingming,2206.0
2258,2599,zhuo jun,2207.0


In [17]:
#get nulls

def get_null(x):
    return x.isnull().sum()

In [21]:
get_null(df_rank_info)

id          0
name        0
position    0
dtype: int64

In [ ]:
personal_info.head()

In [ ]:
master_info.isnull().sum()